In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
import logging
import os
import shutil
import json
import transformers
from transformers import AutoModel, AutoTokenizer, AutoConfig, AutoModelForMaskedLM
from transformers import DataCollatorWithPadding
from datasets import Dataset,load_dataset, load_from_disk
from transformers import TrainingArguments, Trainer
from datasets import load_metric, disable_progress_bar
from sklearn.metrics import mean_squared_error
from datasets import load_dataset
import torch

In [48]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
DATA_DIR = './data/'

In [16]:
ability = 'completeness'

In [17]:
# with open(DATA_DIR+'train/'+CFG.ability+'_train.json', "r") as f:    
#     data_train = json.load(f)
# with open(DATA_DIR+'test/'+CFG.ability+'_test.json', "r") as f:    
#     data_test = json.load(f)

data = load_dataset('json',data_files={'train':DATA_DIR+'train/'+ability+'_train.json','test':DATA_DIR+'test/'+ability+'_test.json'})
#test = load_dataset('json',data_files=DATA_DIR+'test/'+CFG.ability+'_test.json')

Found cached dataset json (/home/work/.cache/huggingface/datasets/json/default-c0bf04b7476fff20/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
train = data['train'].shuffle(seed=42)
test = data['test'].shuffle(seed=42)

Loading cached shuffled indices for dataset at /home/work/.cache/huggingface/datasets/json/default-c0bf04b7476fff20/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-d0ba2f994ae91126.arrow
Loading cached shuffled indices for dataset at /home/work/.cache/huggingface/datasets/json/default-c0bf04b7476fff20/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-0aec500ee8ba0c81.arrow


In [19]:
for d in test:
    if d['completeness']==0:
        print('!')

In [151]:
train[0]

{'grammar': 5.0,
 'question': 'Premise: "yeah i can imagine it wasn\'t like that when i first bought my house uh this gentleman that i was talking to when i did buy my house told me that\'s what happened to him" If this premise is true, what does that tell us about whether it entails the hypothesis "That happened to me, too."?',
 'text': 'Premise: "yeah i can imagine it wasn\'t like that when i first bought my house uh this gentleman that i was talking to when i did buy my house told me that\'s what happened to him" If this premise is true, what does that tell us about whether it entails the hypothesis "That happened to me, too."? [SEP] If the premise is true and the person was told by a gentleman that the described situation happened to him, it suggests that there is a possibility that the same thing could have happened to the speaker as well. However, it does not conclusively prove or entail that the same thing actually did happen to the speaker. Rather, it implies that the speaker\'

In [152]:
def score_to_word(score):
    if score>4:
        return "excellent"
    elif score>3:
        return "good"
    elif score>2:
        return "average"
    elif score>1:
        return "bad"
    elif score>0:
        return "terrible"
def prompt_tokenize(example):
    score_word = score_to_word(example[ability])
    prompted_text = f"Qusetion : {example['question']} [SEP] Response : {example['response']} [SEP] Based on this response, the {ability} level of the response is [MASK]."
    #return tokenizer(prompted_text, return_tensors='pt',padding=True,truncation=True)
    example['prompted_text'] = prompted_text
    example['label'] = score_word
    return example



In [153]:
# train_encoding = tokenizer(
#     train['text'],
#     return_tensors='pt',
#     padding=True,
#     truncation=True
# )
# test_encoding = tokenizer(
#     test['text'],
#     return_tensors='pt',
#     padding=True,
#     truncation=True
# )

train_prompted = train.map(prompt_tokenize)

test_prompted = test.map(prompt_tokenize)

Loading cached processed dataset at /home/work/.cache/huggingface/datasets/json/default-2edb139edbd8ead8/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-0e5109515e394150.arrow


Map:   0%|          | 0/2727 [00:00<?, ? examples/s]

In [154]:
train_prompted.to_json(f'/home/work/deeptext/yys/redpen/data/train/{ability}_prompt_train.json')

Creating json from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

49634614

In [155]:
test_prompted.to_json(f'/home/work/deeptext/yys/redpen/data/test/{ability}_prompt_test.json')

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

5501332

In [156]:
train_prompted[0]

{'grammar': 5.0,
 'question': 'Premise: "yeah i can imagine it wasn\'t like that when i first bought my house uh this gentleman that i was talking to when i did buy my house told me that\'s what happened to him" If this premise is true, what does that tell us about whether it entails the hypothesis "That happened to me, too."?',
 'text': 'Premise: "yeah i can imagine it wasn\'t like that when i first bought my house uh this gentleman that i was talking to when i did buy my house told me that\'s what happened to him" If this premise is true, what does that tell us about whether it entails the hypothesis "That happened to me, too."? [SEP] If the premise is true and the person was told by a gentleman that the described situation happened to him, it suggests that there is a possibility that the same thing could have happened to the speaker as well. However, it does not conclusively prove or entail that the same thing actually did happen to the speaker. Rather, it implies that the speaker\'

In [13]:
train_encoding = tokenizer(
    train_prompted['prompted_text'],
    return_tensors='pt',
    padding=True,
    truncation=True
)
test_encoding = tokenizer(
    test_prompted['prompted_text'],
    return_tensors='pt',
    padding=True,
    truncation=True
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
tokenizer.decode(train_encoding['input_ids'][0])

'[CLS] Qusetion : Premise: "yeah i can imagine it wasn\'t like that when i first bought my house uh this gentleman that i was talking to when i did buy my house told me that\'s what happened to him" If this premise is true, what does that tell us about whether it entails the hypothesis "That happened to me, too."?[SEP] Response : If the premise is true and the person was told by a gentleman that the described situation happened to him, it suggests that there is a possibility that the same thing could have happened to the speaker as well. However, it does not conclusively prove or entail that the same thing actually did happen to the speaker. Rather, it implies that the speaker\'s situation might be similar to the gentleman\'s based on the information they received.[SEP] Based on this response, the readability level of the answer is[MASK].[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]

In [15]:
import torch
from torch.utils.data import Dataset


class PromptDataset(Dataset):
    def __init__(self, encodings):
        self.encoding = encodings

    def __getitem__(self, idx):
        data = {key: val[idx] for key, val in self.encoding.items()}
        

        return data

    def __len__(self):
        return len(self.encoding)

In [29]:
train_set = PromptDataset(train_encoding)
test_set = PromptDataset(test_encoding)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
device

device(type='cuda')

In [30]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)

Some weights of DebertaV2ForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
indexed_tokens = tokenizer.convert_tokens_to_ids(dummy)
tokens_tensor = torch.tensor([indexed_tokens])


In [37]:
train_prompted['prompted_text'][2]

'Qusetion : What is the version of the following sentence with correct punctuation?  Title Nyt fra Schaeffer   [SEP] Response : The New from Schaeffer  "New" should be capitalized.  A:  The proper punctuation would be "The New From Schaeffer". The first word is a proper noun, so it needs to be in all caps.   [SEP] Based on this response, the readability level of the answer is [MASK].'

In [39]:
dummy = tokenizer(train_prompted['prompted_text'][2], return_tensors='pt')

mask_pos = torch.where(dummy['input_ids'] == tokenizer.mask_token_id)[0][0]

In [41]:
exc=tokenizer.convert_tokens_to_ids('excellent')
good=tokenizer.convert_tokens_to_ids('good')
avg= tokenizer.convert_tokens_to_ids('average')
poor=tokenizer.convert_tokens_to_ids('poor')
terr = tokenizer.convert_tokens_to_ids('terrible')

dummy = tokenizer(train_prompted['prompted_text'][2], return_tensors='pt')
mask_pos = torch.where(dummy['input_ids']==tokenizer.mask_token_id)[0][0]
predictions = model(**dummy)[0]
pred=predictions[0, mask_pos][[exc,good,avg,poor,terr]]
prob=torch.nn.functional.softmax(pred, dim=0)
lossFunc = torch.nn.CrossEntropyLoss()
loss=lossFunc(prob.unsqueeze(0), torch.tensor([label]))
loss.backward()
optimizer.step()

In [42]:
prob

tensor([0.0011, 0.0193, 0.1602, 0.8177, 0.0018], grad_fn=<SoftmaxBackward0>)

In [ ]:
from tqdm.notebook import tqdm
from datasets import load_metric

def train(epoch, model, dataloader, optimizer, device):
    model.to(device)

    m1 = load_metric('mse')

    for e in range(1, epoch+1):
        total_loss = 0.
        preds = []
        labels = []
        progress_bar = tqdm(dataloader)
        for data in progress_bar:
            #data = {k:v.to(device) for k, v in data.items()}
            output = model(**data)
            
            current_loss = output.loss
            total_loss += current_loss.item()  # .item()을 사용하여 스칼라 값을 얻음
            
            preds += list(output.logits.squeeze().detach().cpu().numpy())
            labels += list(data['labels'].detach().cpu().numpy())

            optimizer.zero_grad()
            current_loss.backward()
            optimizer.step()

            progress_bar.set_description(f'TRAIN - EPOCH {e} | current-loss: {current_loss:.4f}')
        
        rmse = m1.compute(predictions=preds, references=labels, squared=False)
        avg = total_loss / len(dataloader.dataset)

        print('='*64)
        print(f"TRAIN - EPOCH {e} | LOSS: {avg:.4f} RMSE: {rmse}")
        print('='*64)

In [ ]:
from torch.optim import AdamW

# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
train(1, model, train_loader, optimizer, device)


TRAIN - EPOCH 1 |:   0%|          | 0/32 [00:00<?, ?it/s]

TRAIN - EPOCH 1 | LOSS: 0.0322 RMSE: {'mse': 0.996620854595321}


TRAIN - EPOCH 2 |:   0%|          | 0/32 [00:00<?, ?it/s]

TRAIN - EPOCH 2 | LOSS: 0.0292 RMSE: {'mse': 0.9494141741943081}


TRAIN - EPOCH 3 |:   0%|          | 0/32 [00:00<?, ?it/s]

TRAIN - EPOCH 3 | LOSS: 0.0251 RMSE: {'mse': 0.8896607963716799}


TRAIN - EPOCH 4 |:   0%|          | 0/32 [00:00<?, ?it/s]

TRAIN - EPOCH 4 | LOSS: 0.0174 RMSE: {'mse': 0.7434175833629258}


TRAIN - EPOCH 5 |:   0%|          | 0/32 [00:00<?, ?it/s]

TRAIN - EPOCH 5 | LOSS: 0.0127 RMSE: {'mse': 0.6357503323154271}


In [ ]:
from tqdm.notebook import tqdm
from datasets import load_metric

def evaluate(model, dataloader, device):
    model.to(device)

    m1 = load_metric('mse')

    total_loss = 0.
    preds = []
    labels = []
    progress_bar = tqdm(dataloader, desc=f'EVAL |')
    for data in progress_bar:
        data = {k:v.to(device) for k, v in data.items()}
        with torch.no_grad():
            output = model(**data)
        
        current_loss = output.loss
        total_loss += current_loss.item()  # .item()을 사용하여 스칼라 값을 얻음
        
        preds += list(output.logits.squeeze().detach().cpu().numpy())
        labels += list(data['labels'].detach().cpu().numpy())

        progress_bar.set_description(f'EVAL | current-loss: {current_loss:.4f}')
    
    rmse = m1.compute(predictions=preds, references=labels, squared=False)
    avg = total_loss / len(dataloader.dataset)

    print('='*64)
    print(f"EVAL | LOSS: {avg:.4f} RMSE: {rmse}")
    print('='*64)

In [ ]:
evaluate(model, test_loader, device)


EVAL |:   0%|          | 0/32 [00:00<?, ?it/s]

EVAL | LOSS: 0.0322 RMSE: {'mse': 1.0024117614225598}
